In [1]:
import requests
import os
import json


In [ ]:
# check if /datasets/gutenberg/dataset_books_with_metadata.json exists, if so, load it

# fetch the text of the books from the Gutenberg dataset (too large to upload to GitHub)
with open(f"./datasets/train_100M/gutenberg.train", "r", encoding="utf-8") as f:
    all_books = f.read()
lines = all_books.split("\n")

# split up the books
texts = {}
beginning_indices = []
for i in range(len(lines)):
    if lines[i].startswith("= = = "):
        beginning_indices.append(i)

for i in range(len(beginning_indices) - 1):
    gutenberg_id = lines[beginning_indices[i]][8:].split()[0]
    if not gutenberg_id.isdigit():
        print("Id-length: ", len(gutenberg_id), "for id:", gutenberg_id)
        continue
    text = " ".join(lines[(beginning_indices[i] + 1):(beginning_indices[i + 1] - 1)])
    texts[gutenberg_id] = text
print(f"Found {len(texts)} books in the Gutenberg dataset.")
print("Example book text:", texts.get("52018", "")[:500])  # Print first 500 characters of a sample book

books_with_metadata = {} # id -> { 'text': str, 'author': str, 'title': str, 'subjects': list, 'bookshelves': list }

# if metadata is locally available, load it
if os.path.exists("./datasets/gutenberg/dataset_books_with_metadata.json"):
    with open("./datasets/gutenberg/dataset_books_with_metadata.json", "r", encoding="utf-8") as f:
        books_with_metadata = json.load(f)
    print(f"Loaded {len(books_with_metadata)} books with metadata from local file.")
    print("Example book metadata:", books_with_metadata.get("52018", {}))

# otherwise, fetch metadata from Gutendex
else: 
    # Fetch metadata for each book from Gutendex
    count = 0
    for gutenberg_id, text in texts.items():
        count += 1
        if count % 50 == 0:
            print(f"Processing book ID {gutenberg_id}... ({count}/{len(texts)})")
        try:
            response = requests.get(f"https://gutendex.com/books/{gutenberg_id}", timeout=30)
            response.raise_for_status()
            metadata = response.json()

            authors = metadata.get("authors", [])
            if not authors:
                author = "Unknown"
            else:
                author = authors[0].get("name", "Unknown")
            
            books_with_metadata[gutenberg_id] = {
                
                'author': author,
                'title': metadata["title"],
                'subjects': metadata["subjects"],
                'bookshelves': metadata["bookshelves"]
            }
        except requests.exceptions.RequestException as e:
            print(f"Failed to get metadata for book {gutenberg_id}: {e}")
        except Exception as e:
            print(f"Error processing book {gutenberg_id}: {e}")
    with open("./datasets/gutenberg/dataset_books_with_metadata.json", "w", encoding="utf-8") as f:
        json.dump(books_with_metadata, f, ensure_ascii=False, indent=4)
    print(len(books_with_metadata), "books with metadata found.")
    print("Example book metadata:", books_with_metadata.get(52018, {}))

for gutenberg_id, book in books_with_metadata.items():
    if gutenberg_id in texts:
        book['text'] = texts[gutenberg_id]


In [ ]:
all_bookshelves = set()
all_subjects = set()
bookshelve_counts = {}
bookshelve_word_counts = {}
subject_counts = {}
subject_word_counts = {}
for book in books_with_metadata.values():
    word_count = len(book["text"].split())
    for bookshelve in book["bookshelves"]:
        all_bookshelves.add(bookshelve)
        if bookshelve in bookshelve_counts:
            bookshelve_counts[bookshelve] += 1
            bookshelve_word_counts[bookshelve] += word_count
        else:
            bookshelve_counts[bookshelve] = 1
            bookshelve_word_counts[bookshelve] = word_count
    for subject in book["subjects"]:
        all_subjects.add(subject)
        if subject in subject_counts:
            subject_counts[subject] += 1
            subject_word_counts[subject] += word_count
        else:
            subject_counts[subject] = 1
            subject_word_counts[subject] = word_count

print(f"Total unique bookshelves: {len(all_bookshelves)}")
print(f"Total unique subjects: {len(all_subjects)}")


In [ ]:

bookshelf_list = sorted(list(all_bookshelves))
subject_list = sorted(list(all_subjects))
# print 10 most common bookshelves and subjects with their counts
print("\nMost common bookshelves:")
sorted_bookshelves = sorted(bookshelve_counts.items(), key=lambda x: x[1], reverse=True)
for bookshelf, count in sorted_bookshelves[:10]:
    print(f"{bookshelf}: {count} books")
print("\nMost common subjects:")
sorted_subjects = sorted(subject_counts.items(), key=lambda x: x[1], reverse=True)
for subject, count in sorted_subjects[:10]:
    print(f"{subject}: {count} books")

# print 10 longest bookshelves and subjects with their total word counts
print("\nLongest bookshelves:")
sorted_bookshelves_by_length = sorted(bookshelve_word_counts.items(), key=lambda x: x[1], reverse=True)
for bookshelf, word_count in sorted_bookshelves_by_length[:10]:
    print(f"{bookshelf}: {word_count} words")
print("\nLongest subjects:")
sorted_subjects_by_length = sorted(subject_word_counts.items(), key=lambda x: x[1], reverse=True)
count = 1
for subject, word_count in sorted_subjects_by_length:
    if word_count < 500_000:
        break
    print(f"{count}: {subject}: {word_count} words")
    count += 1



In [ ]:
# create a mapping from subjects to books
subject_to_book_map = {}
for subject in all_subjects:
    subject_to_book_map[subject] = []
for book_id, book in books_with_metadata.items():
    for subject in book["subjects"]:
        subject_to_book_map[subject].append(book_id)

# look at all pairs of subject, and check how many books they have in common
subject_pairs = {}
for i in range(len(subject_list)):
    for j in range(i + 1, len(subject_list)):
        subject1 = subject_list[i]
        subject2 = subject_list[j]
        common_books = set(subject_to_book_map[subject1]) & set(subject_to_book_map[subject2])
        if common_books:
            subject_pairs[(subject1, subject2)] = len(common_books)
print("\nMost common subject pairs:")
sorted_subject_pairs = sorted(subject_pairs.items(), key=lambda x: x[1], reverse=True)
for (subject1, subject2), count in sorted_subject_pairs[:10]:
    print(f"{subject1} & {subject2}: {count} common books")

In [ ]:
# takes in a list of tuples (gutenberg_id, name) a filename and a word limit
# writes file with the books that have a word count less than the limit to 
# ./datasets/gutenberg/genres/filename.train
def create_dataset_from_list(books, filename, word_limit):
    if os.path.exists(f"./datasets/gutenberg/genres/{filename}.train"):
        os.remove(f"./datasets/gutenberg/genres/{filename}.train")
    word_count = 0
    for gutenberg_id, name in books:
        try:
            response = requests.get(f"https://www.gutenberg.org/cache/epub/{gutenberg_id}/pg{gutenberg_id}.txt", timeout=30)
            response.raise_for_status()
            text = response.text
            print(f"Processing book {gutenberg_id} - {name}... ({len(text.split())} words)")
            # print(f"First 100 characters: {text[:100]}")
            if word_count + len(text.split()) > word_limit:
                print(f"Word limit reached. Stopping.")
                # write the remaining number of words to hit the word limit exactly
                text = " ".join(text.split()[:word_limit - word_count])
                with open(f"./datasets/gutenberg/genres/{filename}.train", "a", encoding="utf-8") as f:
                    f.write(f"= = = {gutenberg_id} {name}\n")
                    f.write(text + "\n")
                print(f"Wrote {word_count + len(text.split())} words to {filename}.train with word limit {word_limit}.")
                break
            word_count += len(text.split())
            with open(f"./datasets/gutenberg/genres/{filename}.train", "a", encoding="utf-8") as f:
                f.write(f"= = = {gutenberg_id} {name}\n")
                f.write(text + "\n")
        except requests.exceptions.RequestException as e:
            print(f"Failed to get book {gutenberg_id}: {e}")
        except Exception as e:
            print(f"Error processing book {gutenberg_id}: {e}")
    print(f"Total words written to {filename}.train: {word_count}")

In [ ]:
sci_fi_fantasy = [
    (55, "The Wonderful Wizard of Oz"),
    (36, "The War of the Worlds"),
    (20000, "Twenty Thousand Leagues Under the Sea"),
    (7477, "The Book of Wonder"),
    (20782, "Triplanetary"),
    (10002, "The House on the Borderland"),
    (35, "The Time Machine"),
    (18857, "A Journey to the Centre of the Earth"),
    (1250, "Anthem"),
    (159, "The Island of Doctor Moreau"),
    (21279, "2 B R 0 2 B"),
    (8395, "The Gods of Pegāna"),
    (11, "Alice's Adventures in Wonderland"),   
    (7506, "The Steam Man of the Prairies"),              
    (289, "The Night Land"),                              
    (1230, "The Lost World"),                            
    (22615, "The Princess Nobody: A Tale of Fairyland"),  
    (1505, "The First Men in the Moon"),                
    (12163, "The Sleeper Awakes"),                      
    (3479, "The Metal Monster"),                       
    (17355, "The Runaway Skyscraper"),                  
    (829, "Gulliver's Travels"),
    (12, "Through the Looking‑Glass"),
    (780, "The War in the Air"),           
    (765, "The Moon Pool"),               
    (1013, "The First Men in the Moon"),   
]

create_dataset_from_list(sci_fi_fantasy, "1M_sci-fi_fantasy", 1_000_000)

Processing book 55 - The Wonderful Wizard of Oz... (42687 words)
First 500 characters: ﻿The Project Gutenberg eBook of The Wonderful Wizard of Oz
    
This ebook is for the use of anyon
Processing book 36 - The War of the Worlds... (63114 words)
First 500 characters: ﻿The Project Gutenberg eBook of The War of the Worlds
    
This ebook is for the use of anyone any
Failed to get book 20000: 404 Client Error: Not Found for url: https://www.gutenberg.org/cache/epub/20000/pg20000.txt
Processing book 7477 - The Book of Wonder... (25906 words)
First 500 characters: ﻿The Project Gutenberg eBook of The Book of Wonder
    
This ebook is for the use of anyone anywhe
Processing book 20782 - Triplanetary... (60200 words)
First 500 characters: ﻿The Project Gutenberg eBook of Triplanetary
    
This ebook is for the use of anyone anywhere in 
Processing book 10002 - The House on the Borderland... (53976 words)
First 500 characters: ﻿The Project Gutenberg eBook of The House on the Borderland
    
This